In [30]:
import json
import os
from collections import defaultdict

import gensim
from gensim.models.doc2vec import Doc2Vec
from gensim.summarization import summarize

import math
import operator
import numpy as np
from nltk import word_tokenize,sent_tokenize,pos_tag


In [31]:
def read_data(file_name):
    with open(file_name,'r',encoding="utf8") as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [32]:
totalvocab, w2v = read_data("D:/Deep learning/pretrainedmodels/Glove vectors/glove.6B.50d.txt")

Total Words in DataSet: 400000


In [5]:
json_data=open("D:/Hacks/Wehack3/test data/compresseddict.json").read()

compresseddict = json.loads(json_data)

In [6]:
with open("D:/Hacks/Wehack3/test data/vocab.txt", 'r',encoding='UTF-8') as f:
    vocab = [line.rstrip('\n') for line in f]

In [7]:
len(vocab),len(compresseddict.keys())

(942, 942)

In [33]:
w2v['that']

array([ 0.88387  , -0.14199  ,  0.13566  ,  0.098682 ,  0.51218  ,
        0.49138  , -0.47155  , -0.30742  ,  0.01963  ,  0.12686  ,
        0.073524 ,  0.35836  , -0.60874  , -0.18676  ,  0.78935  ,
        0.54534  ,  0.1106   , -0.2923   ,  0.059041 , -0.69551  ,
       -0.18804  ,  0.19455  ,  0.32269  , -0.49981  ,  0.306    ,
       -2.3902   , -0.60749  ,  0.37107  ,  0.078912 , -0.23896  ,
        3.839    , -0.20355  , -0.35613  , -0.69185  , -0.17497  ,
       -0.35323  ,  0.10598  , -0.039303 ,  0.015701 ,  0.038279 ,
       -0.35283  ,  0.44882  , -0.16534  ,  0.31579  ,  0.14963  ,
       -0.071277 , -0.53506  ,  0.52711  , -0.20148  ,  0.0095952])

In [ ]:
w2v['taiwan']

In [36]:
q="brazil that which"#stockpile->ingredient
q=word_tokenize(q.lower())

for i in range(0,len(q)):
    replace=""
    mindist=1000
    if q[i].lower() not in vocab:
        vec1=w2v[q[i]]
        for j in vocab:
            if j in totalvocab:
                vec2=w2v[j]
                dist = np.linalg.norm(vec1-vec2)
                if dist<mindist:
                    mindist=dist
                    replace=j
        q[i]=replace


x=pos_tag(q)
x

[('mexico', 'NN'), ('that', 'WDT'), ('which', 'WDT')]

In [37]:
directory="D:/Hacks/Wehack3//test data/test_sid/"
namesofdoc=os.listdir(directory)

alpha=0.80
totaldocs=len(namesofdoc)

inv_indx = defaultdict(list)

In [38]:
######## SEARCH #######
dictofdoc={}
listofdoc=[]

for i in namesofdoc:
    dictofdoc[i]=0


for i in namesofdoc:
    dictofdoc[i]=0
    listofdoc.append(i)

for i in x: 
    info=compresseddict[i[0].lower()]
    #finding number of documents query word is in
    numberofdocs=0
    numofdocsnocontext=0
    docs_counted=[]
    docs_nocontext_counted=[]
    listofdoc2=[]
    for j in info:     #j has POStag,docn,number
        listofdoc2.append(j[1])
        if j[0]==i[1]: #matches context
            if j[1] not in docs_counted: #to see whether or not the doc is already counted
                docs_counted.append(j[1])
                numberofdocs=numberofdocs+1
        else:
            if j[1] not in docs_counted: #to see whether or not the doc is already counted
                docs_nocontext_counted.append(j[1])
                numofdocsnocontext=numofdocsnocontext+1
    listofdoc=list(set(listofdoc).intersection(set(listofdoc2)))
            
            
    
    for j in info:     #j has POStag,docn,number
        try:
            if j[0]==i[1]: #matches context
                dictofdoc[j[1]]=dictofdoc[j[1]]+(alpha*j[2]*math.log10((totaldocs+1)/numberofdocs))

            else: #context not matched
                noncontext_match=j[2]
                dictofdoc[j[1]]=dictofdoc[j[1]]+((1-alpha)*j[2]*math.log10((totaldocs+1)/numofdocsnocontext))
        except:
            continue


for i in listofdoc:
    dictofdoc[i]=dictofdoc[i]*15

In [39]:
sortedd = sorted(dictofdoc.items(), key=operator.itemgetter(1))

In [40]:
sortedd

[('hackathon.txt', 0.07752801040644514),
 ('japan_stock.txt', 0.07752801040644514),
 ('modi-japan-trip.txt', 0.07752801040644514),
 ('modi.txt', 0.07752801040644514),
 ('usa.txt', 0.21732201127364886),
 ('programming.txt', 0.3183520069376301),
 ('Japan.txt', 0.5779837768065879),
 ('ainme.txt', 0.7096072645122777),
 ('choclate.txt', 9.198030429265861)]

In [13]:
listtop5=[]
for i in range(1,len(sortedd)):
    listtop5.append(sortedd[-i][0])
    if i==5:
        break

In [14]:
listtop5

['ainme.txt', 'Japan.txt', 'choclate.txt', 'programming.txt', 'usa.txt']

In [72]:
#for images calssified with tessearct, DO NOT RUN!!


sendtonid={}
for i in listtop5:
    try:
        with open(directory+i, 'r',encoding='UTF-8') as f:
            s=f.read()
    except:
        with open(directory+i, 'r') as f:
            s=f.read()
    listofimg=[]
    for j in images:
        if j.startswith(i[:-4]):
            listofimg.append(j)
    if len(listofimg)==0:
        listofimg="k.jpg"
    sendtonid[i]=[summarize(s),listofimg,i]

NameError: name 'images' is not defined